# راهنمای جامع داده‌های پردازش شده MovieLens

## مقدمه
این نوت‌بوک برای آشنایی تیم توسعه با داده‌های پردازش شده و آماده‌سازی شده برای مدل‌های مختلف یادگیری ماشین طراحی شده است.

### محتویات:
1. **بارگذاری داده‌ها**: نحوه دسترسی به داده‌های پردازش شده
2. **داده‌های رگرسیون**: پیش‌بینی امتیاز فیلم‌ها
3. **داده‌های طبقه‌بندی**: تشخیص فیلم‌های محبوب
4. **داده‌های خوشه‌بندی**: گروه‌بندی کاربران و فیلم‌ها
5. **قوانین انجمنی**: کشف الگوهای رفتاری
6. **ماتریس کاربر-آیتم**: سیستم توصیه‌گر

---

In [1]:
# وارد کردن کتابخانه‌های مورد نیاز
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pickle
import gzip
from scipy import sparse
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import warnings
warnings.filterwarnings('ignore')

# تنظیمات فونت فارسی برای matplotlib
plt.rcParams['font.family'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# رنگ‌های زیبا برای نمودارها
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7', '#DDA0DD', '#98D8C8', '#F7DC6F']

print("کتابخانه‌ها با موفقیت بارگذاری شدند ✅")

کتابخانه‌ها با موفقیت بارگذاری شدند ✅


## 1. بارگذاری داده‌های پردازش شده

In [3]:
def load_all_data():
    """بارگذاری تمام داده‌های پردازش شده"""
    data = {}
    
    # بارگذاری ماتریس کاربر-آیتم
    try:
        data['user_item_matrix'] = sparse.load_npz('./data/processed/user_item_matrix.npz')
        print("✅ ماتریس کاربر-آیتم بارگذاری شد")
    except Exception as e:
        print(f"❌ خطا در بارگذاری ماتریس: {e}")
    
    # بارگذاری دیتاست‌های ML
    try:
        with gzip.open('./data/processed/ml_ready_datasets.pkl.gz', 'rb') as f:
            data['ml_datasets'] = pickle.load(f)
        print("✅ دیتاست‌های یادگیری ماشین بارگذاری شدند")
    except Exception as e:
        print(f"❌ خطا در بارگذاری ML datasets: {e}")
    
    # بارگذاری ویژگی‌های پیشرفته
    try:
        with gzip.open('./data/processed/hyper_features.pkl.gz', 'rb') as f:
            data['features'] = pickle.load(f)
        print("✅ ویژگی‌های پیشرفته بارگذاری شدند")
    except Exception as e:
        print(f"❌ خطا در بارگذاری features: {e}")
    
    # بارگذاری ویژگی‌های کاربران و فیلم‌ها از parquet
    try:
        data['user_features'] = pd.read_parquet('./data/processed/user_features.parquet')
        print("✅ ویژگی‌های کاربران بارگذاری شد")
    except:
        pass
    
    try:
        data['movie_features'] = pd.read_parquet('data/processed/movie_features.parquet')
        print("✅ ویژگی‌های فیلم‌ها بارگذاری شد")
    except:
        pass
        
    return data

# بارگذاری داده‌ها
print("🔄 در حال بارگذاری داده‌ها...")
data = load_all_data()
print("\n✅ بارگذاری کامل شد!")

🔄 در حال بارگذاری داده‌ها...
❌ خطا در بارگذاری ماتریس: [Errno 2] No such file or directory: './data/processed/user_item_matrix.npz'
❌ خطا در بارگذاری ML datasets: [Errno 2] No such file or directory: './data/processed/ml_ready_datasets.pkl.gz'
❌ خطا در بارگذاری features: [Errno 2] No such file or directory: './data/processed/hyper_features.pkl.gz'

✅ بارگذاری کامل شد!


## 2. بررسی ساختار داده‌های بارگذاری شده

In [ ]:
# نمایش اطلاعات کلی
print("📊 ساختار داده‌های بارگذاری شده:\n")

if 'ml_datasets' in data:
    print("🎯 دیتاست‌های یادگیری ماشین:")
    for task_name, task_data in data['ml_datasets'].items():
        if task_data is not None:
            print(f"\n  📌 {task_name}:")
            if isinstance(task_data, dict):
                for key, value in task_data.items():
                    if hasattr(value, 'shape'):
                        print(f"    - {key}: شکل {value.shape}")
                    elif isinstance(value, list):
                        print(f"    - {key}: {len(value)} آیتم")
                    elif isinstance(value, dict):
                        print(f"    - {key}: {value}")
                    else:
                        print(f"    - {key}: {type(value).__name__}")

if 'user_item_matrix' in data:
    matrix = data['user_item_matrix']
    print(f"\n📊 ماتریس کاربر-آیتم:")
    print(f"  - ابعاد: {matrix.shape[0]:,} کاربر × {matrix.shape[1]:,} فیلم")
    print(f"  - تعداد امتیازات: {matrix.nnz:,}")
    print(f"  - پراکندگی: {(1 - matrix.nnz / (matrix.shape[0] * matrix.shape[1])) * 100:.2f}%")

## 3. داده‌های رگرسیون - پیش‌بینی امتیاز فیلم‌ها 📈

این بخش برای توسعه‌دهندگانی که روی **پیش‌بینی امتیاز (rating)** کار می‌کنند.

In [ ]:
if 'ml_datasets' in data and 'regression' in data['ml_datasets']:
    reg_data = data['ml_datasets']['regression']
    
    if reg_data and 'X' in reg_data:
        X_reg = reg_data['X']
        y_reg = reg_data['y']
        
        print("🎯 اطلاعات دیتاست رگرسیون:\n")
        print(f"تعداد نمونه‌ها: {X_reg.shape[0]:,}")
        print(f"تعداد ویژگی‌ها: {X_reg.shape[1]}")
        print(f"\nبازه امتیازات: {y_reg.min():.1f} تا {y_reg.max():.1f}")
        print(f"میانگین امتیازات: {y_reg.mean():.2f}")
        print(f"انحراف معیار: {y_reg.std():.2f}")
        
        # نمودار توزیع امتیازات
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        
        # توزیع امتیازات
        axes[0].hist(y_reg, bins=20, color=colors[0], alpha=0.7, edgecolor='black')
        axes[0].set_title('توزیع امتیازات هدف', fontsize=14, pad=10)
        axes[0].set_xlabel('امتیاز')
        axes[0].set_ylabel('تعداد')
        axes[0].grid(True, alpha=0.3)
        
        # نمودار جعبه‌ای
        axes[1].boxplot(y_reg, vert=True, patch_artist=True,
                       boxprops=dict(facecolor=colors[1], alpha=0.7),
                       medianprops=dict(color='red', linewidth=2))
        axes[1].set_title('نمودار جعبه‌ای امتیازات', fontsize=14, pad=10)
        axes[1].set_ylabel('امتیاز')
        axes[1].grid(True, alpha=0.3)
        
        # توزیع تجمعی
        sorted_ratings = np.sort(y_reg)
        p = np.arange(len(sorted_ratings)) / len(sorted_ratings)
        axes[2].plot(sorted_ratings, p, color=colors[2], linewidth=2)
        axes[2].set_title('توزیع تجمعی امتیازات', fontsize=14, pad=10)
        axes[2].set_xlabel('امتیاز')
        axes[2].set_ylabel('احتمال تجمعی')
        axes[2].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # نمایش نام ویژگی‌ها
        if 'feature_names' in reg_data:
            print("\n📋 ویژگی‌های موجود (نمونه):")
            features = reg_data['feature_names']
            for i in range(min(10, len(features))):
                print(f"  {i+1}. {features[i]}")
            if len(features) > 10:
                print(f"  ... و {len(features)-10} ویژگی دیگر")
        
        # نمونه کد برای استفاده
        print("\n💡 نمونه کد برای استفاده:")
        print("""```python
# بارگذاری داده‌ها
X_train = reg_data['X']
y_train = reg_data['y']

# تقسیم به train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# مدل رگرسیون
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# ارزیابی
from sklearn.metrics import mean_squared_error, r2_score
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print(f'MSE: {mse:.3f}, R²: {r2:.3f}')
```""")

## 4. داده‌های طبقه‌بندی - تشخیص فیلم‌های محبوب 🎯

این بخش برای توسعه‌دهندگانی که روی **طبقه‌بندی دودویی** (فیلم خوب/بد) کار می‌کنند.

In [ ]:
if 'ml_datasets' in data and 'classification' in data['ml_datasets']:
    clf_data = data['ml_datasets']['classification']
    
    if clf_data and 'X' in clf_data:
        X_clf = clf_data['X']
        y_clf = clf_data['y']
        
        print("🎯 اطلاعات دیتاست طبقه‌بندی:\n")
        print(f"تعداد نمونه‌ها: {X_clf.shape[0]:,}")
        print(f"تعداد ویژگی‌ها: {X_clf.shape[1]}")
        
        # توازن کلاس‌ها
        if 'class_balance' in clf_data:
            balance = clf_data['class_balance']
            print(f"\nتوازن کلاس‌ها:")
            print(f"  - فیلم‌های محبوب (امتیاز ≥ 4): {balance.get('positive', balance.get(1, 0)):.1%}")
            print(f"  - فیلم‌های معمولی (امتیاز < 4): {balance.get('negative', balance.get(0, 0)):.1%}")
        else:
            # محاسبه دستی توازن
            unique, counts = np.unique(y_clf, return_counts=True)
            balance_dict = dict(zip(unique, counts))
            total = sum(counts)
            
        # نمودارها
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        
        # نمودار دایره‌ای توازن کلاس‌ها
        if 'class_balance' in clf_data:
            labels = ['محبوب (≥4)', 'معمولی (<4)']
            sizes = [balance.get('positive', balance.get(1, 0)), 
                    balance.get('negative', balance.get(0, 0))]
            axes[0].pie(sizes, labels=labels, autopct='%1.1f%%', 
                       colors=[colors[2], colors[3]], startangle=90)
        else:
            labels = [f'کلاس {c}' for c in unique]
            axes[0].pie(counts, labels=labels, autopct='%1.1f%%', 
                       colors=colors[:len(unique)], startangle=90)
        axes[0].set_title('توزیع کلاس‌ها', fontsize=14, pad=20)
        
        # نمودار میله‌ای
        unique_classes, class_counts = np.unique(y_clf, return_counts=True)
        axes[1].bar(unique_classes, class_counts, color=[colors[4], colors[5]])
        axes[1].set_title('تعداد نمونه‌ها در هر کلاس', fontsize=14, pad=10)
        axes[1].set_xlabel('کلاس')
        axes[1].set_ylabel('تعداد')
        axes[1].set_xticks(unique_classes)
        axes[1].set_xticklabels(['معمولی', 'محبوب'])
        axes[1].grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        plt.show()
        
        # نمونه کد
        print("\n💡 نمونه کد برای استفاده:")
        print("""```python
# بارگذاری داده‌ها
X = clf_data['X']
y = clf_data['y']

# تقسیم داده‌ها
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# مدل طبقه‌بندی
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# ارزیابی
from sklearn.metrics import classification_report, confusion_matrix
predictions = model.predict(X_test)
print(classification_report(y_test, predictions, 
                          target_names=['معمولی', 'محبوب']))
```""")

## 5. داده‌های خوشه‌بندی - گروه‌بندی کاربران و فیلم‌ها 🎨

این بخش برای توسعه‌دهندگانی که روی **خوشه‌بندی** و **تحلیل گروه‌ها** کار می‌کنند.

In [ ]:
# خوشه‌بندی کاربران
if 'ml_datasets' in data and 'clustering_users' in data['ml_datasets']:
    user_cluster_data = data['ml_datasets']['clustering_users']
    
    if user_cluster_data and 'X' in user_cluster_data:
        X_users = user_cluster_data['X']
        
        print("👥 اطلاعات خوشه‌بندی کاربران:\n")
        print(f"تعداد کاربران: {X_users.shape[0]:,}")
        print(f"تعداد ویژگی‌ها: {X_users.shape[1]}")
        
        if 'feature_names' in user_cluster_data:
            print("\nویژگی‌های کاربران:")
            for i, feat in enumerate(user_cluster_data['feature_names'][:5]):
                print(f"  {i+1}. {feat}")
        
        # تجسم با t-SNE
        if X_users.shape[0] > 1000:
            sample_indices = np.random.choice(X_users.shape[0], 1000, replace=False)
            X_sample = X_users[sample_indices]
        else:
            X_sample = X_users
        
        # نرمال‌سازی
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_sample)
        
        # t-SNE
        print("\n🔄 در حال محاسبه t-SNE...")
        tsne = TSNE(n_components=2, random_state=42, perplexity=30)
        X_tsne = tsne.fit_transform(X_scaled)
        
        plt.figure(figsize=(10, 8))
        scatter = plt.scatter(X_tsne[:, 0], X_tsne[:, 1], 
                            c=range(len(X_tsne)), cmap='viridis', 
                            alpha=0.6, s=50)
        plt.title('نمایش دوبعدی کاربران با t-SNE', fontsize=16, pad=20)
        plt.xlabel('بعد اول')
        plt.ylabel('بعد دوم')
        plt.colorbar(scatter, label='شاخص کاربر')
        plt.grid(True, alpha=0.3)
        plt.show()

# خوشه‌بندی فیلم‌ها
if 'ml_datasets' in data and 'clustering_movies' in data['ml_datasets']:
    movie_cluster_data = data['ml_datasets']['clustering_movies']
    
    if movie_cluster_data and 'X' in movie_cluster_data:
        X_movies = movie_cluster_data['X']
        
        print("\n🎬 اطلاعات خوشه‌بندی فیلم‌ها:\n")
        print(f"تعداد فیلم‌ها: {X_movies.shape[0]:,}")
        print(f"تعداد ویژگی‌ها: {X_movies.shape[1]}")
        
        # نمونه کد
        print("\n💡 نمونه کد برای خوشه‌بندی:")
        print("""```python
# خوشه‌بندی کاربران
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# نرمال‌سازی داده‌ها
scaler = StandardScaler()
X_scaled = scaler.fit_transform(user_cluster_data['X'])

# تعیین تعداد بهینه خوشه‌ها
inertias = []
K_range = range(2, 11)
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)

# انتخاب k=5 و خوشه‌بندی نهایی
kmeans = KMeans(n_clusters=5, random_state=42)
user_clusters = kmeans.fit_predict(X_scaled)

# تحلیل خوشه‌ها
for i in range(5):
    cluster_users = np.where(user_clusters == i)[0]
    print(f'خوشه {i}: {len(cluster_users)} کاربر')
```""")

## 6. قوانین انجمنی - کشف الگوهای رفتاری 🔍

این بخش برای توسعه‌دهندگانی که روی **کشف الگوها** و **قوانین انجمنی** کار می‌کنند.

In [ ]:
if 'ml_datasets' in data and 'association_rules' in data['ml_datasets']:
    assoc_data = data['ml_datasets']['association_rules']
    
    if assoc_data and 'transactions' in assoc_data:
        transactions = assoc_data['transactions']
        
        print("🔍 اطلاعات قوانین انجمنی:\n")
        print(f"تعداد تراکنش‌ها (کاربران): {len(transactions):,}")
        print(f"حداقل پشتیبانی (min_support): {assoc_data.get('min_support', 'نامشخص')}")
        print(f"حداقل اطمینان (min_confidence): {assoc_data.get('min_confidence', 'نامشخص')}")
        
        # آمار تراکنش‌ها
        trans_lengths = [len(t) for t in transactions if t]
        if trans_lengths:
            print(f"\nآمار تراکنش‌ها:")
            print(f"  - میانگین فیلم‌ها در هر تراکنش: {np.mean(trans_lengths):.1f}")
            print(f"  - حداقل: {np.min(trans_lengths)}")
            print(f"  - حداکثر: {np.max(trans_lengths)}")
            
            # نمودار توزیع طول تراکنش‌ها
            plt.figure(figsize=(10, 6))
            plt.hist(trans_lengths, bins=50, color=colors[6], alpha=0.7, edgecolor='black')
            plt.title('توزیع تعداد فیلم‌های محبوب هر کاربر', fontsize=14, pad=10)
            plt.xlabel('تعداد فیلم‌ها')
            plt.ylabel('تعداد کاربران')
            plt.grid(True, alpha=0.3)
            plt.show()
        
        # نمونه کد
        print("\n💡 نمونه کد برای کشف قوانین:")
        print("""```python
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# تبدیل تراکنش‌ها به فرمت مناسب
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_transactions = pd.DataFrame(te_ary, columns=te.columns_)

# کشف آیتم‌های مکرر
frequent_items = apriori(df_transactions, 
                        min_support=0.01, 
                        use_colnames=True)

# استخراج قوانین
rules = association_rules(frequent_items, 
                         metric="confidence", 
                         min_threshold=0.5)

# نمایش قوانین برتر
print("قوانین با بالاترین اطمینان:")
print(rules.nlargest(10, 'confidence')[['antecedents', 'consequents', 
                                        'support', 'confidence', 'lift']])
```""")

## 7. ماتریس کاربر-آیتم - سیستم توصیه‌گر 🎬

این بخش برای توسعه‌دهندگانی که روی **سیستم‌های توصیه‌گر** کار می‌کنند.

In [ ]:
if 'user_item_matrix' in data:
    matrix = data['user_item_matrix']
    
    print("🎬 اطلاعات ماتریس کاربر-آیتم:\n")
    print(f"ابعاد: {matrix.shape[0]:,} کاربر × {matrix.shape[1]:,} فیلم")
    print(f"تعداد امتیازات ثبت شده: {matrix.nnz:,}")
    print(f"درصد پراکندگی: {(1 - matrix.nnz / (matrix.shape[0] * matrix.shape[1])) * 100:.2f}%")
    
    # توزیع امتیازات
    ratings = matrix.data
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # هیستوگرام امتیازات
    axes[0].hist(ratings, bins=np.arange(0.5, 6, 0.5), 
                color=colors[0], alpha=0.7, edgecolor='black')
    axes[0].set_title('توزیع امتیازات در ماتریس', fontsize=14, pad=10)
    axes[0].set_xlabel('امتیاز')
    axes[0].set_ylabel('تعداد')
    axes[0].grid(True, alpha=0.3)
    
    # آمار کاربران
    user_counts = np.array((matrix != 0).sum(axis=1)).flatten()
    axes[1].hist(user_counts[user_counts > 0], bins=50, 
                color=colors[1], alpha=0.7, edgecolor='black')
    axes[1].set_title('توزیع تعداد امتیازات هر کاربر', fontsize=14, pad=10)
    axes[1].set_xlabel('تعداد امتیاز')
    axes[1].set_ylabel('تعداد کاربران')
    axes[1].set_yscale('log')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # نمونه کد
    print("\n💡 نمونه کد برای سیستم توصیه‌گر:")
    print("""```python
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

# محاسبه شباهت بین کاربران
user_similarity = cosine_similarity(matrix, dense_output=False)

# پیش‌بینی امتیاز برای کاربر i و فیلم j
def predict_rating(user_id, movie_id, k=50):
    # یافتن k کاربر مشابه
    user_sim = user_similarity[user_id].toarray().flatten()
    similar_users = np.argsort(user_sim)[-k-1:-1][::-1]
    
    # محاسبه میانگین وزن‌دار
    numerator = 0
    denominator = 0
    
    for similar_user in similar_users:
        if matrix[similar_user, movie_id] > 0:
            numerator += user_sim[similar_user] * matrix[similar_user, movie_id]
            denominator += user_sim[similar_user]
    
    if denominator > 0:
        return numerator / denominator
    else:
        return matrix[user_id].mean()  # میانگین امتیازات کاربر

# توصیه فیلم‌ها برای یک کاربر
def recommend_movies(user_id, n_recommendations=10):
    # فیلم‌هایی که کاربر ندیده
    user_ratings = matrix[user_id].toarray().flatten()
    unseen_movies = np.where(user_ratings == 0)[0]
    
    # پیش‌بینی امتیاز برای فیلم‌های ندیده
    predictions = []
    for movie_id in unseen_movies[:1000]:  # محدود به 1000 فیلم برای سرعت
        pred = predict_rating(user_id, movie_id)
        predictions.append((movie_id, pred))
    
    # مرتب‌سازی و انتخاب بهترین‌ها
    predictions.sort(key=lambda x: x[1], reverse=True)
    return predictions[:n_recommendations]
```""")

## 8. ویژگی‌های کاربران و فیلم‌ها 📊

بررسی ویژگی‌های استخراج شده برای کاربران و فیلم‌ها

In [ ]:
# ویژگی‌های کاربران
if 'user_features' in data:
    user_features = data['user_features']
    print("👥 ویژگی‌های کاربران:\n")
    print(f"تعداد کاربران: {len(user_features):,}")
    print(f"تعداد ویژگی‌ها: {len(user_features.columns)}")
    print("\nنمونه ویژگی‌ها:")
    for col in user_features.columns[:10]:
        print(f"  - {col}")
    
    # نمایش آمار توصیفی
    print("\nآمار توصیفی (5 ویژگی اول):")
    print(user_features.iloc[:, :5].describe().round(2))

# ویژگی‌های فیلم‌ها
if 'movie_features' in data:
    movie_features = data['movie_features']
    print("\n\n🎬 ویژگی‌های فیلم‌ها:\n")
    print(f"تعداد فیلم‌ها: {len(movie_features):,}")
    print(f"تعداد ویژگی‌ها: {len(movie_features.columns)}")
    print("\nنمونه ویژگی‌ها:")
    for col in movie_features.columns[:10]:
        print(f"  - {col}")

## 9. خلاصه و راهنمای استفاده 📚

### خلاصه داده‌های موجود:

1. **رگرسیون**: پیش‌بینی امتیاز دقیق فیلم‌ها (0.5 تا 5)
2. **طبقه‌بندی**: تشخیص فیلم‌های محبوب (امتیاز ≥ 4) از معمولی
3. **خوشه‌بندی کاربران**: گروه‌بندی کاربران بر اساس رفتار
4. **خوشه‌بندی فیلم‌ها**: گروه‌بندی فیلم‌ها بر اساس ویژگی‌ها
5. **قوانین انجمنی**: کشف الگوهای مشترک در انتخاب فیلم‌ها
6. **ماتریس کاربر-آیتم**: برای سیستم‌های توصیه‌گر مبتنی بر فیلترینگ مشارکتی

### نکات مهم:

- همه داده‌ها از قبل پردازش و آماده شده‌اند
- ویژگی‌ها نرمال‌سازی شده و آماده استفاده در مدل‌ها هستند
- برای جلوگیری از overfitting، از تقسیم train/test استفاده کنید
- در صورت نیاز به ویژگی‌های اضافی، با تیم پیش‌پردازش هماهنگ کنید

## 10. ذخیره‌سازی و صدور داده‌ها 💾

In [ ]:
# تابع کمکی برای ذخیره داده‌ها
def save_dataset_for_task(task_name, output_format='csv'):
    """ذخیره دیتاست یک تسک خاص"""
    
    if 'ml_datasets' not in data or task_name not in data['ml_datasets']:
        print(f"❌ دیتاست {task_name} یافت نشد!")
        return
    
    task_data = data['ml_datasets'][task_name]
    if not task_data:
        print(f"❌ دیتاست {task_name} خالی است!")
        return
    
    # ایجاد دایرکتوری خروجی
    import os
    os.makedirs('exported_data', exist_ok=True)
    
    if 'X' in task_data and 'y' in task_data:
        # تبدیل به DataFrame
        X_df = pd.DataFrame(task_data['X'])
        if 'feature_names' in task_data:
            X_df.columns = task_data['feature_names']
        
        y_df = pd.DataFrame(task_data['y'], columns=['target'])
        
        # ترکیب X و y
        full_df = pd.concat([X_df, y_df], axis=1)
        
        # ذخیره
        if output_format == 'csv':
            full_df.to_csv(f'exported_data/{task_name}_dataset.csv', index=False)
            print(f"✅ دیتاست {task_name} در فایل exported_data/{task_name}_dataset.csv ذخیره شد")
        elif output_format == 'parquet':
            full_df.to_parquet(f'exported_data/{task_name}_dataset.parquet', index=False)
            print(f"✅ دیتاست {task_name} در فایل exported_data/{task_name}_dataset.parquet ذخیره شد")

# مثال استفاده
print("📥 برای ذخیره دیتاست‌ها از دستورات زیر استفاده کنید:\n")
print("save_dataset_for_task('regression', 'csv')    # ذخیره دیتاست رگرسیون")
print("save_dataset_for_task('classification', 'parquet')  # ذخیره دیتاست طبقه‌بندی")

---

### 🎯 موفق باشید!

این نوت‌بوک توسط تیم پیش‌پردازش داده آماده شده است. در صورت نیاز به اطلاعات بیشتر یا ویژگی‌های اضافی، با ما در ارتباط باشید.

**نسخه**: 1.0  
**تاریخ**: ۱۴۰۳/۰۹/۲۰